### Sift Based

In [1]:
import cv2
import numpy as np
import json
from datetime import timedelta

In [ ]:
# Caminhos para os arquivos
logo_path = "../target/logo_target.png"
video_path = '../videos/15s-canto-direito-rotacionada.mp4'
print(video_path)

# Carregar a logotipo target
logo_image = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
if logo_image is None:
    raise IOError("Não foi possível carregar a imagem da logotipo.")

# Iniciar a captura de vídeo
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Não foi possível abrir o vídeo.")

# Inicializar SIFT
sift = cv2.SIFT_create()

# Encontrar keypoints e descritores na logotipo
kp_logo, des_logo = sift.detectAndCompute(logo_image, None)

# Configurar o FLANN Matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)  # Número de verificações

flann = cv2.FlannBasedMatcher(index_params, search_params)

# Parâmetros de detecção
MIN_MATCH_COUNT = 10  # Número mínimo de correspondências para considerar uma detecção
logo_present = False
start_time = None
end_time = None
detections = []

fps = cap.get(cv2.CAP_PROP_FPS)

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Fim do vídeo

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar keypoints e descritores no frame
    kp_frame, des_frame = sift.detectAndCompute(frame_gray, None)

    if des_frame is not None:
        # Encontrar correspondências usando FLANN
        matches = flann.knnMatch(des_logo, des_frame, k=2)

        # Aplicar o ratio test de Lowe
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        if len(good_matches) > MIN_MATCH_COUNT:
            # Encontrou a logotipo no frame
            if not logo_present:
                logo_present = True
                start_time = frame_count / fps
        else:
            if logo_present:
                logo_present = False
                end_time = frame_count / fps
                detections.append({
                    "start": str(timedelta(seconds=start_time)),
                    "end": str(timedelta(seconds=end_time))
                })
    else:
        if logo_present:
            logo_present = False
            end_time = frame_count / fps
            detections.append({
                "start": str(timedelta(seconds=start_time)),
                "end": str(timedelta(seconds=end_time))
            })

    frame_count += 1

# Se o vídeo terminar com a logotipo presente
if logo_present:
    end_time = frame_count / fps
    detections.append({
        "start": str(timedelta(seconds=start_time)),
        "end": str(timedelta(seconds=end_time))
    })

cap.release()

# Salvar em um arquivo JSON
output = {"detections": detections}
#with open('detecoes_logo.json', 'w') as json_file:
    #json.dump(output, json_file, indent=4)

#print("Detecções salvas em 'detecoes_logo.json'.")
print(output)

In [7]:
detection = {'start': detections[0]['start'], 'end': detections[-1]['end']}

In [ ]:
detection

In [3]:
import cv2
import numpy as np
import json
from datetime import timedelta

def detectar_logo_no_video2(video_path, logo_path, output_frame_path, min_match_count=10, ratio_test=0.7):
    """
    Detecta a presença de uma logotipo em um vídeo e salva o primeiro frame onde a logo foi detectada.
    Retorna os timestamps de aparição e desaparecimento.

    :param video_path: Caminho para o arquivo de vídeo.
    :param logo_path: Caminho para a imagem da logotipo target.
    :param output_json_path: Caminho para salvar o arquivo JSON com os resultados.
    :param output_frame_path: Caminho para salvar o primeiro frame onde a logotipo foi detectada.
    :param min_match_count: Número mínimo de correspondências para considerar uma detecção.
    :param ratio_test: Valor do ratio test de Lowe para filtrar correspondências.
    """
    # Carregar a logotipo target em escala de cinza
    logo_image = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
    if logo_image is None:
        raise IOError("Não foi possível carregar a imagem da logotipo.")

    # Iniciar a captura de vídeo
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Não foi possível abrir o vídeo.")

    # Inicializar o detector SIFT
    sift = cv2.SIFT_create()

    # Detectar keypoints e descritores na logotipo
    kp_logo, des_logo = sift.detectAndCompute(logo_image, None)
    if des_logo is None:
        raise ValueError("Não foi possível encontrar descritores na logotipo.")

    # Configurar o FLANN Matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)  # Número de verificações
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Parâmetros de detecção
    logo_present = False
    start_time = None
    end_time = None
    detections = []
    frame_saved = False  # Verifica se o primeiro frame já foi salvo

    # Obter frames por segundo (FPS) do vídeo
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        raise ValueError("FPS do vídeo é 0, verifique o arquivo de vídeo.")

    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Fim do vídeo

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar keypoints e descritores no frame
        kp_frame, des_frame = sift.detectAndCompute(frame_gray, None)

        if des_frame is not None:
            # Encontrar correspondências usando FLANN
            try:
                matches = flann.knnMatch(des_logo, des_frame, k=2)
            except cv2.error as e:
                print(f"Erro durante a correspondência de características: {e}")
                matches = []

            # Aplicar o ratio test de Lowe
            good_matches = []
            for m_n in matches:
                if len(m_n) != 2:
                    continue
                m, n = m_n
                if m.distance < ratio_test * n.distance:
                    good_matches.append(m)

            if len(good_matches) > min_match_count:
                # Encontrou a logotipo no frame
                if not logo_present:
                    logo_present = True
                    start_time = frame_count / fps

                    # Salvar o primeiro frame onde a logotipo foi detectada
                    if not frame_saved:
                        cv2.imwrite(output_frame_path, frame)
                        frame_saved = True
                        print(f"Primeiro frame salvo em '{output_frame_path}'.")

            else:
                if logo_present:
                    logo_present = False
                    end_time = frame_count / fps
                    detections.append({
                        "start": str(timedelta(seconds=start_time)),
                        "end": str(timedelta(seconds=end_time))
                    })
        else:
            if logo_present:
                logo_present = False
                end_time = frame_count / fps
                detections.append({
                    "start": str(timedelta(seconds=start_time)),
                    "end": str(timedelta(seconds=end_time))
                })

        frame_count += 1

    # Se o vídeo terminar com a logotipo presente
    if logo_present:
        end_time = frame_count / fps
        detections.append({
            "start": str(timedelta(seconds=start_time)),
            "end": str(timedelta(seconds=end_time))
        })

    cap.release()

    # Estruturar o resultado
    output = {"detections": detections}

    # Salvar em um arquivo JSON
    #with open(output_json_path, 'w') as json_file:
        #json.dump(output, json_file, indent=4, ensure_ascii=False)

    return output


In [ ]:
logo_path = "../target/logo_target.png"
video_path = '../videos/15s-canto-esquerdo-pequeno.mp4'
detectar_logo_no_video2(video_path, logo_path, 'frame_to_mask_canto_esquerdo_pequeno.png')

### Using masks

In [1]:
import cv2
import numpy as np
import json

def detectar_logo_com_sift_e_mascara(video_path, logo_path, mask_path, min_match_count=10):
    # Inicializar SIFT
    sift = cv2.SIFT_create()

    # Carregar o template (logo) e sua máscara
    logo = cv2.imread(logo_path, 0)
    mask = cv2.imread(mask_path, 0)

    # Detectar keypoints e descritores na logo usando a máscara
    kp_logo, des_logo = sift.detectAndCompute(logo, mask)

    # Configurar FLANN para correspondência de keypoints
    index_params = dict(algorithm=1, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Carregar vídeo
    cap = cv2.VideoCapture(video_path)

    frame_count = 0
    fps = cap.get(cv2.CAP_PROP_FPS)  # Obter a taxa de frames por segundo do vídeo

    logo_present = False
    logo_detected = False
    detections = []
    start_time = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar keypoints e descritores no frame atual
        kp_frame, des_frame = sift.detectAndCompute(frame_gray, None)

        if des_frame is not None:
            # Correspondência de descritores
            matches = flann.knnMatch(des_logo, des_frame, k=2)

            # Aplicar ratio test para selecionar boas correspondências
            good_matches = []
            for m, n in matches:
                if m.distance < 0.75 * n.distance:
                    good_matches.append(m)

            # Se o número de boas correspondências for suficiente, considerar que a logo foi detectada
            if len(good_matches) > min_match_count:
                if not logo_present:
                    logo_present = True
                    start_time = frame_count / fps

                logo_detected = True
            else:
                if logo_present and logo_detected:
                    # A logo desapareceu
                    end_time = frame_count / fps
                    detections.append({
                        "start": start_time,
                        "end": end_time
                    })
                    logo_present = False
                    logo_detected = False

        frame_count += 1

    cap.release()

    # Caso a logo esteja presente até o final do vídeo, adicionar a última detecção
    if logo_present and logo_detected:
        end_time = frame_count / fps
        detections.append({
            "start": start_time,
            "end": end_time
        })

    return detections

In [8]:
# Exemplo de uso
video_path = '../videos/15s-canto-direito-rotacionada.mp4'
logo_path = "../target/logo_target.png"
mask_path = '/masks/mask_canto_direito_rotacionada.png'

result = detectar_logo_com_sift_e_mascara(video_path, logo_path, mask_path)
print(result)

[{'start': 5.0, 'end': 5.6}, {'start': 5.633333333333334, 'end': 5.666666666666667}, {'start': 5.733333333333333, 'end': 5.766666666666667}, {'start': 5.8, 'end': 6.766666666666667}, {'start': 6.8, 'end': 6.833333333333333}, {'start': 6.933333333333334, 'end': 6.966666666666667}, {'start': 7.1, 'end': 7.166666666666667}, {'start': 7.4, 'end': 7.433333333333334}, {'start': 7.466666666666667, 'end': 7.633333333333334}, {'start': 7.8, 'end': 7.833333333333333}, {'start': 8.633333333333333, 'end': 8.8}, {'start': 8.833333333333334, 'end': 8.9}, {'start': 8.933333333333334, 'end': 8.966666666666667}, {'start': 9.0, 'end': 9.066666666666666}, {'start': 9.1, 'end': 9.133333333333333}, {'start': 14.5, 'end': 14.6}]


In [6]:
detection = {'start': result[0]['start'], 'end': result[-1]['end']}
print(detection)

{'start': 4.266666666666667, 'end': 9.833333333333334}
